# Use the embeding and learnable posencoding as BERTEncoder as Decoder

In [1]:
# coding = utf-8

import torch
from torch import nn
import os
import data_load_for_Transformer as dlft
import BERT_pretraining_single
import BERTModel
import Utility
import imp
imp.reload(BERTModel)

<module 'BERTModel' from 'e:\\ML\\chatbot\\BERTChatbot\\BERTModel.py'>

In [2]:
print("Initiating the hyperparameters...")
num_hiddens, num_layers, dropout, batch_size, num_steps = 32, 2, 0.1, 128, 16
lr, num_epochs, device = 0.001, 2000, Utility.try_gpu()
ffn_num_input, ffn_num_hiddens, num_heads = 32, 64, 4
key_size, query_size, value_size = 32, 32, 32
norm_shape = [32]

Initiating the hyperparameters...


In [3]:
train_iter, vocab = dlft.load_data_xhj_for_Transformer(batch_size, num_steps)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\RuiLi\AppData\Local\Temp\jieba.cache
Using 'word' as tokens
Loading model cost 0.556 seconds.
Prefix dict has been built succesfully.
total training size= 3102


In [4]:
print(vocab.to_tokens([0,1,2,3,4,5,6,7,8,9]))

['<unk>', '<pad>', 'mask2', '<bos>', '<eos>', '你', '我', '的', '了', '是']


In [5]:
print("Rebuilding the Model...")
bert = BERTModel.BERTModel(len(vocab), num_hiddens=num_hiddens, norm_shape=norm_shape, 
    ffn_num_input=ffn_num_input, ffn_num_hiddens=ffn_num_hiddens, num_heads=num_heads, 
    num_layers=num_layers, dropout=dropout, key_size=key_size, query_size=query_size, 
    value_size=value_size, hid_in_features=value_size, mlm_in_features=value_size, 
    nsp_in_features=value_size)
bert = BERT_pretraining_single.load_bert(bert, device)
# for v in bert.parameters():
#     v.requires_grad = False
encoder = BERTModel.BERTEncoder(bert, hid_in_features=value_size, num_outputs=value_size)
decoder = BERTModel.BERTDecoder(
    len(vocab), num_hiddens=num_hiddens, norm_shape=norm_shape, 
    ffn_num_input=ffn_num_input, ffn_num_hiddens=ffn_num_hiddens, num_heads=num_heads, 
    num_layers=num_layers, dropout=dropout, key_size=key_size, query_size=query_size, 
    value_size=value_size, hid_in_features=value_size, mlm_in_features=value_size, 
    nsp_in_features=value_size)
net = BERTModel.EncoderDecoder(encoder, decoder)

try:
    checkpoint_prefix = os.path.join("model_data/model_bert_ed.pt")
    checkpoint = torch.load(checkpoint_prefix)
    net.load_state_dict(checkpoint['model_state_dict'])
except Exception as e:
    print("Can not load the model with error:", e)

print("Ready to working...")
def predict(src_sentence):
    return BERTModel.predict_seq2seq(net, src_sentence, vocab, vocab, num_steps,
                    device, save_attention_weights=False)


Rebuilding the Model...
Can not load the model with error: [Errno 2] No such file or directory: 'model_data/model_bert_ed.pt'
Ready to working...


In [6]:
cp = os.path.join("model_data/model_bert_ed.pt")
BERTModel.train_bert(net, train_iter, lr, num_epochs, batch_size, vocab, device, cp)

KeyboardInterrupt: 